データを加工するノートブック

# 秋野編集

## ライブラリのインポート

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import nltk


## データ読み込み

In [ ]:
# データフォルダの場所を設定
dataPath = "/data"
dataPath

In [ ]:
# データの読み出し
df = pd.read_csv(dataPath + "/train.csv")
scores = df["score"]

## 前処理

In [ ]:
# nltkデータ読み込み
nltk.download('punkt')

In [ ]:
# 各テキストのベクトルを計算する関数
def text_to_vector(text, model):
    vectors = [model.wv[word] for word in text if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

In [ ]:
def get_text_vectors(train_df):
    # テキストとスコアを取得
    texts = train_df["full_text"]
    

    # テキストをトークン化
    tokenized_texts = [word_tokenize(text.lower()) for text in texts]

    # Word2Vecモデルの訓練
    word2vec_model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

    # 各テキストのベクトルを計算
    text_vectors = np.array([text_to_vector(text, word2vec_model) for text in tokenized_texts])

    return text_vectors, word2vec_model

# DataFrameに含まれたテキストデータから、トークン化されたtext_vectorを取得
text_vectors, word2vec_model = get_text_vectors(df)

In [ ]:
text_vectors_df = pd.DataFrame(text_vectors)
text_vectors_df['score'] = df[['score']].copy()
# text_vectors_df

In [ ]:
def get_text_vectors_with_model(train_df, word2vec_model):
    # テキストとスコアを取得
    texts = train_df["full_text"]
    
    # テキストをトークン化
    tokenized_texts = [word_tokenize(text.lower()) for text in texts]

    # 各テキストのベクトルを計算
    text_vectors = np.array([text_to_vector(text, word2vec_model) for text in tokenized_texts])

    return text_vectors

## 学習

In [ ]:
def train_by_randomForest(text_vectors_df, n_estimators, random_state=42):
    feature_columns = [i for i in text_vectors_df.columns if i != "score"]
    train_df = text_vectors_df[feature_columns]
    target = text_vectors_df[["score"]]

    # データを訓練セットをテストセットに分割
    X_train, X_test, y_train, y_test = train_test_split(train_df, target, test_size=0.2, random_state=42)   

    # ランダムフォレスト分類器を訓練
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # テストデータに対する予測
    y_pred = model.predict(X_test)
    kappa_quadratic = cohen_kappa_score(y_test, y_pred, weights="quadratic")
    print("Weighted Kappa 二乗重み付け：", kappa_quadratic)

    return model

model = train_by_randomForest(text_vectors_df, n_estimators=100)  

## 予測とSubmission

In [ ]:
# テストデータの読み出し
test_df = pd.read_csv(dataPath + "/test.csv")

# DataFrameに含まれたテキストデータから、トークン化されたtext_vectorを取得
test_text_vectors = get_text_vectors_with_model(test_df, word2vec_model)

# text_vectorを使って、予測の実行
test_pred = model.predict(test_text_vectors)

In [ ]:
submission_df = test_df[["essay_id"]].copy()
submission_df['score'] = test_pred
submission_df.to_csv('submission.csv',index=False)

# 岡本編集

## 必要なライブラリインポート

In [ ]:
#a
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

## データ確認

In [ ]:
train_df = pd.read_csv('./data/train.csv')
train_df

In [ ]:
train_df.shape

In [ ]:
test_df = pd.read_csv('./data/test.csv')
test_df

In [ ]:
test_df.shape

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.dtypes.to_frame().T

## 特徴量作成

In [ ]:
def processing(df):
    """特徴量作成関数

    特徴量の説明
        text_len:テキストの長さ
        space_count:空白の数
        word_len_avg:一節の平均的な長さ
        I-cnt:”私”という単語の出現頻度

    Args:
        df(pandas.DataFrame):加工したいデータフレーム
    Return:
        pandas.DataFrame:加工後のデータフレーム

    """
    
    df['text_len'] = df.full_text.str.len()
    df['space_count'] = df.full_text.str.count(' ')
    df['word_len_avg'] = (df.text_len - df.space_count) / (df.space_count + 1)
    df['I-cnt'] = df.full_text.str.startswith('I') + df.full_text.str.count('. I ')
    return df

In [ ]:
train_df = processing(train_df)
train_df.head()

In [ ]:
test_df = processing(test_df)
test_df.head()

## テキスト翻訳

In [ ]:
from googletrans import Translator

In [ ]:
translator = Translator()

In [ ]:
def translate_text(text):
    try:
        translated = translator.translate(text,src='en',dest='ja').text
        return translated
    except Exception as e:
            return None

In [ ]:
tqdm.pandas()

In [ ]:
train_df['text_ja'] = train_df.full_text.progress_apply(translate_text)

In [ ]:
train_df.head()

In [ ]:
#翻訳後データフレームをcsv出力
train_df.to_csv('./data/trandlated_df.csv',index=False)

## 単語特徴量作成

### 単語の出現頻度確認

In [ ]:
def check_freq(df):
    vec_count = CountVectorizer()
    vec_count.fit(df.full_text)
    X = vec_count.transform(df.full_text)
    word_df = pd.DataFrame(X.toarray(), columns=vec_count.get_feature_names_out())
    df = pd.concat([df, word_df], axis=1)
    word_df = pd.DataFrame(word_df.sum(axis=0).sort_values(ascending=False).reset_index())
    word_df.columns = ['word', 'count']
    return df,word_df

In [ ]:
#訓練データのテキストを単語に分解
train_df, word_df_train = check_freq(train_df)
display(train_df.head())
display(word_df_train.head())

In [ ]:
#テストデータのテキストを単語に分解
test_df, word_df_test = check_freq(test_df)
display(test_df.head())
display(word_df_test.head())

### ストップワード削除

In [ ]:
#英語のストップワードダウンロード
nltk.download('stopwords')
stop_words = stopwords.words('english')

In [ ]:
stop_words_set = set(stop_words)
def drop_stopword(df,columns_set):
    
    #単語データフレーム内にどれだけストップワードが含まれるのか確認
    columns_set = set(df.word)

    #共通単語抽出
    and_set = columns_set & stop_words_set

    #単語データフレームに存在しなかったストップワード数抽出
    diff_set = stop_words_set - and_set

    #ストップワードの除去、除去前後で矛盾がないか確認
    before_count = len(df)
    df.drop(columns=list(and_set),inplace=True)
    after_count = len(df)
    if len(and_set) - (before_count - after_count) == 0:
        print('処理に問題はありません')
    else:
        print('処理後のレコード数が想定しているレコード数と一致しません')
    

    return df

In [ ]:
tmp_df = drop_stopword(train_df)

## 前処理後データ出力

In [ ]:
train_df.to_csv('./data/processed_train.csv', index=False)

In [ ]:
test_df.to_csv('./data/processed_test.csv', index=False)